<a href="https://colab.research.google.com/github/AsmaaBoudjenane/RAG/blob/main/Rag_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  huggingface_hub
!pip install transformers sentence-transformers faiss-cpu
!pip install gradio
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#AutoTokenizer processes text input, converting it to token IDs suitable for the model.
#AutoModelForCausalLM is used for language generation tasks, creating responses based on a prompt.

aragpt_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-large")
aragpt_model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-large")

# Set a padding token if it doesn't already exist
if aragpt_tokenizer.pad_token is None:
    aragpt_tokenizer.pad_token = aragpt_tokenizer.eos_token
#(end-of-sequence token) as the pad_token, ensuring that inputs with different lengths are padded consistently.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

The repository for aubmindlab/aragpt2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/aubmindlab/aragpt2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_aragpt2.py:   0%|          | 0.00/11.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/aubmindlab/aragpt2-large:
- configuration_aragpt2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

The repository for aubmindlab/aragpt2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/aubmindlab/aragpt2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for aubmindlab/aragpt2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/aubmindlab/aragpt2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling_aragpt2.py:   0%|          | 0.00/83.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/aubmindlab/aragpt2-large:
- modeling_aragpt2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.20G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import gradio as gr
import pandas as pd
import torch
import faiss
import numpy as np


# Load data
file_path = 'legaldata_extracted.xlsx'
df = pd.read_excel(file_path)
print(df)

    case_id    case_title                                   case_description
0   1082906         إثبات  الكلمات األساسیة: إنابة قضائیة - خبرة أجنبیة. ...
1   1070114           بنك  الكلمات األساسیة: عقد قرض– خبرة. المرجع القانو...
2   1070121           بنك  الكلمات األساسیة: قرض – امتیاز – مطالبة فوریة....
3   1076801         تأمین  الكلمات األساسیة: عقد تأمین شامل - ضرر - تعھد ...
4   1082039         تبلیغ  الكلمات األساسیة: شخص مقیم بالخارج – نیابة. ال...
..      ...           ...                                                ...
93     8059  لجنة التعویض  الكلمات األساسیة: أمر إیداع- ر. المرجع القانون...
94     8016  لجنة التعویض  الكلمات األساسیة ر المرجع القانوني: المادة: 13...
95     8109  لجنة التعویض  الكلمات األساسیة: حبس مؤقت غیر مبرر - محبوس قا...
96     8189  لجنة التعویض  الكلمات األساسیة ر المرجع القانوني : المادة: 1...
97  1031961  غرفة اإلتھام  الكلمات األساسیة: انتفاء وجھ الدعوى– إحالة على...

[98 rows x 3 columns]


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re



# Create an Arabic-aware text splitter
def clean_arabic_text(text):
    # Basic cleaning for Arabic text
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()
    return text

# Initialize the text splitter with Arabic-specific separators
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", "!", "؟", ":", "؛"],  # Arabic punctuation
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

# Function to process each text
def chunk_arabic_text(text):
    if not isinstance(text, str):
        return []

    # Clean the text
    text = clean_arabic_text(text)

    # Split the text
    chunks = text_splitter.split_text(text)

    return chunks

# Apply chunking to your dataframe
df['case_chunks'] = df['case_description'].apply(chunk_arabic_text)

# Print example to check results
print("Example chunks from first case:", df['case_chunks'].iloc[0])

Example chunks from first case: ['الكلمات األساسیة: إنابة قضائیة - خبرة أجنبیة. المرجع القانوني : المادة 112 من قانون اإلجراءات المدنیة واإلداریة. المبــدأ: للقاضي أن یصدر، تلقائیا، إنابة قضائیة دولیة ،متى تبین لھ أن التحقیق ضروري للفصل في القضیة المعروضة علیھ وأنھ ال یمكن القیام بھ إال في دولة أجنبیة. إن المحكمـة العـلــیا في جلستھا العلنیة المنعقدة بمقرھا شارع 11 دیسمبر ،1960 األبیار، بن عكنون، الجزائر', '. بعد المداولة القانونیة أصدرت القرار اآلتي نصھ: بناء على المواد 349 إلى 360 و377 إلى 378 و557 إلى 581 من قانون اإلجراءات المدنیة واإلداریة. بعد االطالع على مجموع أوراق ملف الدعوى، وعلى عریضة الطعن بالنقض المودعة بتاریخ 2015/04/12 وعلى مذكرة الرد التي تقدم بھا محامي المطعون ضده. بعد االستماع إلى المستشار المقرر في تالوة تقریره المكتوبوإلى المحامي العام في تقدیم طلباتھ المكتوبة. حیث طلب المدعو )ب', '. حیث طلب المدعو )ب. م( ، بواسطة محامیھ األستاذ دعاس لخضر نقض وإبطال القرار الصادر بتاریخ 2015/01/21 عن مجلس قضاء المدیة یقضي حضوریا نھائیا في الشكل قبول اإلستئناف، وفي الموضوع تأیید الحك

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd

# Load tokenizer and model (use the pre-trained AraBERT or any other model)
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def get_embeddings(chunks):
    if not isinstance(chunks, list):
        return []

    embeddings = []

    # Process chunks in batches
    for text in chunks:
        # Tokenize and encode text
        inputs = tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=512,  # AraBERT's max length
            padding='max_length'
        )

        # Move inputs to same device as model
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Get [CLS] token embedding (better representation than mean)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(sentence_embedding.squeeze())

    return np.array(embeddings)

# Apply to dataframe
df['chunk_embeddings'] = df['case_chunks'].apply(get_embeddings)

# Print shape to verify
print("Shape of first document embeddings:", df['chunk_embeddings'].iloc[0].shape)

Shape of first document embeddings: (12, 768)


In [ ]:
# Flatten all embeddings for FAISS indexing
all_embeddings = np.vstack(df['chunk_embeddings'].explode().to_numpy()).astype('float32')
index = faiss.IndexFlatL2(all_embeddings.shape[1])
index.add(all_embeddings)
# Map each embedding to the corresponding case ID
chunk_embeddings_flat = [embedding for chunk_list in df['chunk_embeddings'] for embedding in chunk_list]
#ink each embedding to its case ID.
case_id_mapping = [case_id for case_id, chunks in zip(df['case_id'], df['chunk_embeddings']) for _ in chunks]

# Create a DataFrame that maps each embedding to a case_id
embedding_df = pd.DataFrame({
    'case_id': case_id_mapping,
    'embedding': chunk_embeddings_flat
})



In [ ]:
# Automatic query generation function
def generate_sample_queries(title, description):
    # Basic sample queries based on title and description
    queries = [
        f"ما هي تفاصيل القضية '{title}'؟",
        f"ما هو الحكم في القضية '{title}'؟",
        f"أخبرني المزيد عن {description[:50]}...",  # Extract keywords or initial part of the description
        f"ما هي القضية المتعلّقة بـ '{title}'؟"
    ]
    return queries

# Apply query generation to each case
df['sample_queries'] = df.apply(lambda x: generate_sample_queries(x['case_title'], x['case_description']), axis=1)
print("Sample queries for the first case:", df['sample_queries'].iloc[10])

Sample queries for the first case: ["ما هي تفاصيل القضية 'شركة'؟", "ما هو الحكم في القضية 'شركة'؟", 'أخبرني المزيد عن الكلمات األساسیة: حلول الشركة - تغییر االسم - الصی...', "ما هي القضية المتعلّقة بـ 'شركة'؟"]


In [ ]:
def retrieve(query, k=3):
    try:
        # Preprocess the query to extract the case title
        case_title_match = re.search(r"القضية\s+'([^']+)'", query)
        case_title = case_title_match.group(1) if case_title_match else None

        if case_title:
            # Look for exact matches in case_title column
            results = df[df['case_title'].str.contains(case_title, na=False, case=False)]

            if not results.empty:
                # Return case details for exact matches
                return " ".join(results['case_description'])

        # If no exact match, fallback to similarity-based search
        query_embedding = get_embeddings([query])[0]
        distances, indices = index.search(np.array([query_embedding]).astype('float32'), k=k)

        # Retrieve case IDs based on closest matches
        retrieved_ids = embedding_df.iloc[indices[0]]['case_id'].unique()
        results = df[df['case_id'].isin(retrieved_ids)]

        if results.empty:
            return ""

        # Concatenate chunks for comprehensive context if available
        context_text = " ".join([" ".join(chunks) for chunks in results['case_chunks']])
        return context_text

    except Exception as e:
        print("Error in retrieve function:", e)
        return ""


In [ ]:
# Generate response using AraGPT model
def generate_response(query, context):
    try:
        # Check if context is empty
        if not context:
            return "لم يتم العثور على سياق مناسب للإجابة على سؤالك."

        input_text = f"سؤال: {query} النص المرجعي: {context}"
        inputs = aragpt_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
        outputs = aragpt_model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask, #Indicates which tokens are meaningful and which are padding.
            max_new_tokens=230,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            pad_token_id=aragpt_tokenizer.eos_token_id
        )
        response = aragpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    except Exception as e:
        print("Error in generate_response function:", e)
        return "تعذر الرد. يرجى المحاولة لاحقاً."


In [ ]:
# Define main function for Gradio
def chat(query):
    retrieved_case = retrieve(query)
    if retrieved_case:
        context_tokens = aragpt_tokenizer(retrieved_case, return_tensors="pt", truncation=False) #processing it for response generation
        token_count = context_tokens['input_ids'].size(1) # ensures that the context won’t exceed the model’s MxL

        # Truncate context if too long
        if token_count > 512:  # Adjust based on model's max token length
            retrieved_case = aragpt_tokenizer.decode(context_tokens['input_ids'][0, :512], skip_special_tokens=True)

        response = generate_response(query, retrieved_case)
    else:
        response = "لم يتم العثور على نتائج متعلقة بالسؤال."
    return response

# Set up Gradio Interface
iface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(label="أدخل سؤالك هنا"),
    outputs=gr.Textbox(label="الرد"),
    title="Chatbot قانوني",
    description="أدخل سؤالك حول القضايا القانونية باللغة العربية وسيتولى الشات بوت مهمة الإجابة.",
)

# Launch the Interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7571c7f3a6179bfa90.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
